<a href="https://colab.research.google.com/github/nhsengland/ds_251_RAG/blob/open_source_rag_clean/open_source_rag_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Using RAG with NHS Conditions - Demo

# Setup

In [ ]:
#clone the github repo from the url below to access relevant files
!git clone https://github.com/nhsengland/ds_251_RAG

Cloning into 'ds_251_RAG'...
remote: Enumerating objects: 326, done.
remote: Counting objects: 100% (167/167), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 326 (delta 87), reused 128 (delta 75), pack-reused 159
Receiving objects: 100% (326/326), 27.96 MiB | 25.61 MiB/s, done.
Resolving deltas: 100% (140/140), done.


install relevant dependencies using pip

In [ ]:
#set working directory to the main repo folder
%cd ds_251_RAG

/content/ds_251_RAG


In [ ]:

# %pip install --upgrade --quiet  transformers langchain_community bitsandbytes langchain accelerate tensorflow==2.15 chromadb unstructured sentence-transformers faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 95.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.1/163.1 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

import relevant modeul

In [ ]:
#import relevant modules
import glob
import os
import pandas as pd
import toml


import src.models as models

from tqdm import tqdm

config = toml.load("config.toml")

if config['DEV_MODE']:
    config['PERSIST_DIRECTORY'] += "/dev"

First we initialise the RAG pipeline - this is an object which links the vector-store, and the LLM, so when you pass a query in it get passed back into the database, and then returns the response.

There are also methods for adding documents to the database

In [ ]:
rag_pipeline = models.RagPipeline(config['EMBEDDING_MODEL'], config['PERSIST_DIRECTORY'], model_type='phi2')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

need to fill the database if it's empty (this might take 5 mins or so the first time, unless you've got a nice graphics card!)

In [ ]:
# Add documents if there are non - if in DEV mode, don't add any more (if it's not empty)
if len(rag_pipeline.vectorstore.get()['documents']) == 0 or (not config['DEV_MODE']):
    rag_pipeline.load_documents()

Using the phi2 model generate a response for the question below with RAG turned off

## Generating Response from Cogstack Questions

<h3> Load in Cogstack QA from Github Repo </h3>

link to cogstack QA data "https://raw.githubusercontent.com/CogStack/OpenGPT/main/data/nhs_uk_full/prepared_generated_data_for_nhs_uk_qa.csv"

In [ ]:
#load processed questions and answers
cogstack_qa = pd.read_csv('src/model_eval/cogstack_qa_data_process.csv')

#select a random sample of questions
sample_qa = cogstack_qa.sample(n = 5, random_state = 999)
sample_qa

,Unnamed: 0,question,answer,reference,short_reference
24400,24400,What can I do if someone with epilepsy has a s...,If the person is in a wheelchair during a seiz...,https://www.nhs.uk/conditions/what-to-do-if-so...,what-to-do-if-someone-has-a-seizure-fit
16010,16010,What is included in the prenatal and postnatal...,The prenatal and postnatal yoga video is desig...,https://www.nhs.uk/conditions/nhs-fitness-stud...,nhs-fitness-studio
8321,8321,Why would someone need dialysis?,People with advanced chronic kidney disease (k...,https://www.nhs.uk/conditions/dialysis/,dialysis
7153,7153,How can I check if a hospital or clinic is reg...,To check if a hospital or clinic registered to...,https://www.nhs.uk/conditions/cosmetic-procedu...,cosmetic-procedures
17219,17219,What happens if my IUD is causing PID symptoms?,"If your IUD is causing PID symptoms, you may b...",https://www.nhs.uk/conditions/pelvic-inflammat...,pelvic-inflammatory-disease-pid


# Testing Phi-2 model - RAG off

In [ ]:
question = sample_qa['question'].values[0]
print(rag_pipeline.llm('Instruction: {} \n\n Output:'.format(question)))


Instruction: What can I do if someone with epilepsy has a seizure while in a wheelchair? 

 Output: If someone with epilepsy has a seizure while in a wheelchair, it is important to follow these steps:

1. Ensure the person's safety: Move any sharp or heavy objects away from the person to prevent injury.

2. Clear the area: Remove any obstacles that may hinder the person's movement during the seizure.

3. Support the person's head: Gently tilt the person's head back slightly to prevent choking on saliva or vomit.

4. Time the seizure: Use a stopwatch or your phone to time the seizure. This information can be helpful for medical professionals.

5. Do not restrain the person: Do not try to restrain the person or hold them down during the seizure. This can cause injury.

6. Keep the person comfortable: Make sure the person is comfortable and not overheated. Remove any tight clothing or accessories.

7. Stay with the person: Stay with the person until the seizure ends. If the seizure lasts 

In [ ]:
question = sample_qa['question'].values[0]

result = rag_pipeline.answer_question(question, model_type = 'phi2', rag=False)

print(result)

KeyboardInterrupt: 

Now try generating the response with RAG turned on

In [ ]:
question = "What are the symptoms of achalasia?"

result = rag_pipeline.answer_question(question, rag=True)

print(result)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Instruction: With this context

docs/achalasia.txt:
Other symptoms include:

bringing back up undigested food
choking and coughing fits
heartburn
chest pain
repeated chest infections
drooling of vomit or saliva
gradual but significant weight loss
Symptoms of achalasia may start at any time of life.

Swallowing problems can also be caused by cancers of the mouth, throat and oesophagus. There’s a link between long-term achalasia and the risk of developing cancer of the oesophagus, but the risk is small.

create a final answer with references ("SOURCES"). If you don't know the answer, just say that you don't know. Don't try to make up an answer. Always return the file name from the context used to answer 
the question

Question: What are the symptoms of achalasia?
Output:

> Finished chain.

> Finished chain.
Instruction: With this context

docs/achalasia.txt:
Other symptoms include:
